In [1]:
%matplotlib nbagg
from planet4.clustering import ClusteringManager
from planet4 import io, markings, plotting
from IPython.display import display
from ipywidgets import FloatText
from pathlib import Path

In [9]:
db = io.DBManager("/Volumes/USB128GB/planet4/2015-11-22_planet_four_classifications_queryable_cleaned.h5")
dbname = db.dbname

In [10]:
obsids = !cat /Users/klay6683/Dropbox/data/planet4/season2_3_image_names.txt

In [11]:
img_name = obsids[0]

In [5]:
def process_image_name(img_name):
    from pathlib import Path
    from planet4.clustering import ClusteringManager
    output_dir = Path('/Users/klay6683/Dropbox/data/planet4/season23_v3')
    output_dir.mkdir(exist_ok=True)
    cm = ClusteringManager(dbname, scope='planet4',
                           output_dir=output_dir,
                           output_format='both')
    cm.cluster_image_name(img_name)
    return img_name

In [6]:
process_image_name(img_name)

'ESP_013113_0985'

In [6]:
from planet4.clustering import ClusteringManager
fnotched_dir = Path('/Users/klay6683/Dropbox/data/planet4/debug')
fnotched_dir.mkdir(exist_ok=True)

In [12]:
def check_processing_image_id(img_id):
    cm = ClusteringManager(dbname, scope='planet4',
                       fnotched_dir=fnotched_dir,
                       output_format='both')
    cm.cluster_image_id(img_id)
    p4id = markings.ImageID(img_id, database_fname=dbname)
    fig, axes = plt.subplots(nrows=2, sharex=True)
    resman = plotting.ResultManager(img_id, fnotched_dir)
    p4id.plot_blotches(ax=axes[0],
                       blotches=resman.blotches(unfnotched=True))
    p4id.plot_fans(ax=axes[1],
                   fans=resman.fans(unfnotched=True))
    cm.apply_fnotch_cut(img_id)
    return cm

In [14]:
cm = check_processing_image_id('APF000036k')

--- Logging error ---
Traceback (most recent call last):
  File "/Users/klay6683/miniconda3/envs/py35/lib/python3.5/logging/__init__.py", line 980, in emit
    msg = self.format(record)
  File "/Users/klay6683/miniconda3/envs/py35/lib/python3.5/logging/__init__.py", line 830, in format
    return fmt.format(record)
  File "/Users/klay6683/miniconda3/envs/py35/lib/python3.5/logging/__init__.py", line 567, in format
    record.message = record.getMessage()
  File "/Users/klay6683/miniconda3/envs/py35/lib/python3.5/logging/__init__.py", line 330, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/klay6683/miniconda3/envs/py35/lib/python3.5/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/klay6683/miniconda3/envs/py35/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/klay6683/miniconda3/envs/py35/lib/python3.5/site-packages/ipykernel/__

TypeError: apply_fnotch_cut() missing 1 required positional argument: 'id_'

In [18]:
resman = plotting.ResultManager("APF000036k", cm.fnotched_dir)

In [23]:
resman.fnotchfile

TypeError: 'PosixPath' object is not callable

In [9]:
cd ~/Dropbox/data/planet4/debug/

/Users/klay6683/Dropbox/data/planet4/debug


In [10]:
ll

total 6448
-rw-r--r--@ 1 klay6683  LASP\Domain Users     8430 Nov 25 16:41 APF000036k_blotches.csv
-rw-r--r--@ 1 klay6683  LASP\Domain Users  1098808 Nov 25 16:41 APF000036k_blotches.hdf
-rw-r--r--@ 1 klay6683  LASP\Domain Users      879 Nov 25 16:41 APF000036k_fans.csv
-rw-r--r--@ 1 klay6683  LASP\Domain Users  1086672 Nov 25 16:41 APF000036k_fans.hdf
-rw-r--r--@ 1 klay6683  LASP\Domain Users     1137 Nov 25 16:41 APF000036k_fnotches.csv
-rw-r--r--@ 1 klay6683  LASP\Domain Users  1086672 Nov 25 16:41 APF000036k_fnotches.hdf


In [25]:
df = pd.read_hdf('./APF000036k_fnotches.hdf')

In [29]:
fc = markings.FnotchContainer.from_df(df)

In [30]:
fc.content[0]

Fnotch value: 0.43

Fan:
x              344.167
y                  164
angle          10.9772
spread         45.4477
distance       50.2048
arm1_x         381.727
arm1_y          156.19
arm2_x         376.083
arm2_y         185.286
image_id    APF000036k
Name: 0, dtype: object

Blotch:
x              370.688
y                  169
angle          57.5947
radius_1        30.456
radius_2        23.467
p1_x           387.009
p1_y           195.358
p2_x           354.366
p2_y           142.642
p3_x           350.875
p3_y           181.576
p4_x             390.5
p4_y           156.424
image_id    APF000036k
Name: 0, dtype: object

In [10]:
from ipyparallel import Client
c = Client()

In [11]:
lbview = c.load_balanced_view()

In [12]:
dview = c.direct_view()

In [13]:
dview.push({'dbname': str(db.dbname)})

<AsyncResult: finished>

In [14]:
image_ids = []
for image_name in obsids:
    image_ids.append(db.get_image_name_markings(image_name)['image_id'].unique())

In [15]:
image_ids = pd.concat([pd.Series(i).astype(str) for i in image_ids])

In [16]:
len(image_ids)

6407

In [11]:
cm = ClusteringManager(dbname, scope='planet4',
                       output_dir=output_dir,
                       output_format='both', cut=0.5)

In [20]:
todo = image_ids[:]
results = lbview.map_async(cm.cluster_image_id, todo)

In [15]:
from iuvs.multitools import nb_progress_display

In [22]:
nb_progress_display(results, todo)

In [25]:
for res in results:
    res

# Applying cut to fnotches

In [17]:
from pathlib import Path
from planet4 import io

In [15]:
output_dir = io.dropbox() / 'season23_by_id_fnotched_0.5/'
input_dir = io.dropbox() / 'season23_by_id'

In [16]:
fnotch_fnames = list(input_dir.glob('*_fnotches.hdf'))

In [18]:
len(fnotch_fnames)

1688

In [19]:
def filter_for_fans(x):
    if isinstance(x, planet4.markings.Fan):
        return x

def filter_for_blotches(x):
    if isinstance(x, planet4.markings.Blotch):
        return x

In [20]:
from planet4 import markings, plotting

def get_newfans_newblotches(fname, input_dir):
    resman = plotting.ResultManager(fname, input_dir)
    df = resman.fnotchdf
    final_clusters = df.apply(markings.Fnotch.from_dataframe, axis=1).\
        apply(lambda x: x.get_marking(0.5))
    newfans = final_clusters[final_clusters.apply(filter_for_fans).notnull()]
    newblotches = final_clusters[final_clusters.apply(filter_for_blotches).notnull()]
    return newfans, newblotches
    
def process_fnotch_fname(fname):
    outpath = output_dir
    outpath.mkdir(exist_ok=True)
    newfans, newblotches = get_newfans_newblotches(fname, input_dir)
    
    if len(newfans) > 0:
        newfans = newfans.apply(lambda x: x.store())
        try:
            completefans = pd.DataFrame(resman.fandf).append(newfans, ignore_index=True)
        except OSError:
            completefans = newfans
    else:
        completefans = resman.fandf
    if len(newblotches) > 0:
        newblotches = newblotches.apply(lambda x: x.store())
        try:
            completeblotches = pd.DataFrame(resman.blotchdf).append(newblotches,
                                                      ignore_index=True)
        except OSError:
            completeblotches = newblotches
    else:
        completeblotches = resman.blotchdf
    completefans.to_hdf(str(outpath / resman.fanfile.name), 'df')
    completeblotches.to_hdf(str(outpath / resman.blotchfile.name), 'df')

In [21]:
image_ids = [i.stem.split('_')[0] for i in fnotch_fnames]

In [22]:
image_ids[0]

'APF000010p'

In [27]:
cm.apply_fnotch_cut(image_ids[0])

AttributeError: ("'Fan' object has no attribute 'image_id'", 'occurred at index 0')

In [23]:
results = lbview.map_async(cm.apply_fnotch_cut, image_ids)

In [24]:
nb_progress_display(results, image_ids)

In [32]:
cm.

0.5

In [25]:
for res in results:
    res

CompositeError: one or more exceptions from call to method: apply_fnotch_cut
[0:apply]: AttributeError: ("'Fan' object has no attribute 'image_id'", 'occurred at index 0')
[1:apply]: PandasError: DataFrame constructor not properly called!
[5:apply]: PandasError: DataFrame constructor not properly called!
[4:apply]: PandasError: DataFrame constructor not properly called!
.... 1683 more exceptions ...

In [13]:
resman = plotting.ResultManager('APF000036k', input_dir)

In [14]:
type(resman.fandf)

pandas.core.frame.DataFrame

In [37]:
resman.blotchdf

,x,y,angle,radius_1,radius_2,p1_x,p1_y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y
0,79.166667,563.833333,44.576761,14.482960,12.153886,89.483035,571.944721,68.850298,555.721945,70.636289,572.490678,87.697044,555.175989
1,588.125000,23.250000,17.049648,18.270330,15.577747,605.592368,5.457102,570.657632,41.042898,583.557601,38.143122,592.692399,8.356878
2,169.125000,457.250000,38.358737,10.662278,10.000000,177.485725,463.784066,160.764275,450.715934,162.919168,465.091406,175.330832,449.408594
3,619.928571,597.428571,70.117456,14.394736,12.399120,624.824121,609.560091,615.033022,585.297052,608.268541,601.645426,631.588602,593.211717
4,498.125000,485.000000,41.206464,14.494004,12.433003,509.029427,479.816097,487.220573,490.183903,489.934457,494.353853,506.315543,475.646147
5,693.500000,459.285714,43.681093,14.507452,11.952017,703.991718,454.981778,683.008282,463.589651,685.245413,467.929355,701.754587,450.642074
6,712.000000,424.333333,27.624995,10.000000,10.000000,720.860014,430.379937,703.139986,418.286730,707.363174,433.193347,716.636826,415.473319
7,705.166667,380.333333,111.055319,16.359207,13.102738,699.289308,365.757036,711.044025,394.909631,692.938746,375.625924,717.394587,385.040743
8,663.916667,387.333333,47.465266,13.699519,11.941306,673.178049,382.746948,654.655284,391.919719,655.117505,395.406098,672.715829,379.260568
9,638.500000,413.250000,85.086544,21.433034,17.324776,640.335759,407.666647,636.664241,418.833353,621.238889,414.733883,655.761111,411.766117
